# __Sudoku Solver__ using Genetic Algorithms
Using genetic algorithms to solve sudokus has several limitations. In fact, these methods often struggle to find the real solution. The goal of this project is to compare the performances of different variants of genetic algorithms. In particular, I will start with a straightforward approach (tournament selection, basic crossover and mutations) to tackle small sudoku grids (such as 4x4 and 6x6). Then, I will consider a harder version of the problem, i.e. solving 9x9 grids. It will be immediately clear that the basic approach previously used will fail to find solutions. Thus, I will explore different variants, by trying several selection methods and exploring the parameter space, and see how the performance changes.


In general, for a $n\times n$ sudoku grid, the goal is to find the unique solution which satisfies the following rules:
1. every row should contain each number from 1 to $n$ exactly once;
2. every columns should contain each number from 1 to $n$ exactly once;
3. each subgrid should contain each number from 1 to $n$ exactly once;

_Disclaimer_: since most of the functions involve some randomness, results could slightly change every time the code is ran. However, general behavior is expected to remain more or less the same. To avoid this issue, one could add random seeds.

## __Useful Imports__

In [1]:
import numpy as np
import random

## __Easy sudoku Implementation__ (4x4 grid)
I start with the easiest sudoku structure, i.e. a 4x4 grid. Throughout the rest of the notebook, I will use the following way to represent sudoku grids: each puzzle will be constructed as a np.array containing the lines of the grid. The cells containing non-zero numbers will be considered fixed, while the cells containing zeroes are the ones we need to modify.

In [2]:
easy_puzzle = np.array([
    [0, 0, 0, 0],
    [2, 0, 4, 3],
    [3, 4, 0, 2],
    [0, 0, 0, 0]]
)

With the previously stated rules in mind, the fitness of a grid can be defined as follows.

In [3]:
def easy_fitness(grid):
    score = 0

    for i in range(4):
        score += len(set(grid[i]))  # row uniqueness
        score += len(set(grid[:, i]))  # column uniqueness
    
    # subgrid uniqueness
    for row in [0, 2]:
        for col in [0, 2]:
            subgrid = grid[row:row+2, col:col+2].flatten()
            score += len(set(subgrid))
            
    return score

In [4]:
easy_fitness(easy_puzzle) 

30

Maximal fitness for a 4x4 grid would be: $$4\times 4+4\times 4+4\times 4 = 48$$
Consider for example the following solution of the easy puzzle, which can be computed by hand.

In [5]:
easy_solution = np.array(
    [[4, 3, 2, 1],
     [2, 1, 4, 3],
     [3, 4, 1, 2],
     [1, 2, 3, 4]
    ]
)

easy_fitness(easy_solution)

48

As expected, the fitness of the solution is 48, thus indicating it is a correct solution. Now let's implement a genetic algorithm with tournament selection to find the correct solution.

### __Genetic Algorithm Implementation (4x4 grid)__

In [6]:
def initialize_population(puzzle, population_size):
    population = []
    for _ in range(population_size):
        grid = puzzle.copy()
        for i in range(4):
            missing_numbers = list(set(range(1, 5)) - set(grid[i]))
            np.random.shuffle(missing_numbers)
            for j in range(4):
                if grid[i][j] == 0:
                    grid[i][j] = missing_numbers.pop()
        population.append(grid)
    return population

In [7]:
def tournament_selection(population, fitnesses, tournament_size):
    selected = random.sample(range(len(population)), tournament_size)
    best = max(selected, key=lambda idx: fitnesses[idx])
    return population[best]

In [8]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, 3)
    child = parent1.copy()
    child[crossover_point] = parent2[crossover_point].copy()
    return child

def mutate(grid, puzzle, mutation_rate):
    if random.random() < mutation_rate:
        row = random.randint(0, 3)
        col1, col2 = random.sample(range(4), 2)

        while puzzle[row][col1] != 0 or puzzle[row][col2] != 0:
            row = random.randint(0, 3)
            col1, col2 = random.sample(range(4), 2)
        grid[row][col1], grid[row][col2] = grid[row][col2], grid[row][col1]
    return grid

In [9]:
def genetic_algorithm(puzzle, population_size, mutation_rate, tournament_size, generations):
    population = initialize_population(puzzle, population_size)
    best_solution = None
    best_fitness = 0
    
    for generation in range(generations):
        fitnesses = [easy_fitness(grid) for grid in population]
        current_best_fitness = max(fitnesses)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_index = fitnesses.index(current_best_fitness)  
            best_solution = population[best_index]  

        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            child = crossover(parent1, parent2)
            child = mutate(child, puzzle, mutation_rate)
            new_population.append(child)
        
        population = new_population  
        print(f"Generation {generation} - Best Fitness: {best_fitness}")

        if best_fitness ==  48:
            break

    return best_solution

In [11]:
solution = genetic_algorithm(easy_puzzle, 100, 0.05, 5, 100)

if np.allclose(solution, easy_solution):
    print('The solution found by the algorithm is equivalent to the solution we had previously found!')


Generation 0 - Best Fitness: 46
Generation 1 - Best Fitness: 46
Generation 2 - Best Fitness: 48
The solution found by the algorithm is equivalent to the solution we had previously found!


Happily, we see that the genetic algorithm approach works for easy sudokus, finding a solution very quickly. For this reason, I will not further explore the parameter space, as I am already satisfied by the obtained results. Now let's consider a slightly harder problem.


## __6x6 Grid__

In [12]:
puzzle = np.array([
    [0, 3, 0, 4, 0, 0],
    [0, 0, 5, 6, 0, 3],
    [0, 0, 0, 1, 0, 0],
    [0, 1, 0, 3, 0, 5],
    [0, 6, 4, 0, 3, 1],
    [0, 0, 1, 0, 4, 6]   
    ]
)

In [13]:
def fitness_6(grid):
    score = 0

    for i in range(6):
        score += len(set(grid[i]))  # row uniqueness
        score += len(set(grid[:, i]))  # column uniqueness
    
    # subgrid uniqueness
    for row in [0, 2, 4]:
        for col in [0, 3]:
            subgrid = grid[row:row+2, col:col+3].flatten()
            score += len(set(subgrid))
            
    return score

In [14]:
def initialize_population(puzzle, population_size):
    population = []
    for _ in range(population_size):
        grid = puzzle.copy()
        for i in range(6):
            missing_numbers = list(set(range(1, 7)) - set(grid[i]))
            np.random.shuffle(missing_numbers)
            for j in range(6):
                if grid[i][j] == 0:
                    grid[i][j] = missing_numbers.pop()
        population.append(grid)
    return population

In [15]:
def tournament_selection(population, fitnesses, tournament_size):
    selected = random.sample(range(len(population)), tournament_size)
    best = max(selected, key=lambda idx: fitnesses[idx])
    return population[best]

In [16]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, 5)
    child = parent1.copy()
    child[crossover_point] = parent2[crossover_point].copy()
    return child

def mutate(grid, puzzle, mutation_rate):
    if random.random() < mutation_rate:
        row = random.randint(0, 5)
        col1, col2 = random.sample(range(6), 2)

        while puzzle[row][col1] != 0 or puzzle[row][col2] != 0:
            row = random.randint(0, 5)
            col1, col2 = random.sample(range(6), 2)
        grid[row][col1], grid[row][col2] = grid[row][col2], grid[row][col1]
    return grid

In [17]:
def genetic_algorithm(puzzle, population_size, mutation_rate, tournament_size, generations):
    population = initialize_population(puzzle, population_size)
    best_solution = None
    best_fitness = 0
    
    for generation in range(generations):
        fitnesses = [fitness_6(grid) for grid in population]
        current_best_fitness = max(fitnesses)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_index = fitnesses.index(current_best_fitness)  
            best_solution = population[best_index]  

        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            child = crossover(parent1, parent2)
            child = mutate(child, puzzle, mutation_rate)
            new_population.append(child)
        
        population = new_population  

        print(f"Generation {generation} - Best Fitness: {best_fitness}")

        if best_fitness ==  108:
            break

    return best_solution

In [23]:
solution = genetic_algorithm(puzzle, 100, 0.1, 5, 100)

print(solution)

Generation 0 - Best Fitness: 100
Generation 1 - Best Fitness: 101
Generation 2 - Best Fitness: 104
Generation 3 - Best Fitness: 106
Generation 4 - Best Fitness: 108
[[1 3 6 4 5 2]
 [2 4 5 6 1 3]
 [6 5 3 1 2 4]
 [4 1 2 3 6 5]
 [5 6 4 2 3 1]
 [3 2 1 5 4 6]]


We see that the algorithm successfully finds a solution for the 6x6 grid as well, quickly enough.

## __9x9 Grids__
Now let's tackle a harder class of sudokus, which are 9x9 grids. Of course, the previously created functions will need to be updated. The maximal fitness will now be:
$$9\times 9 + 9\times 9+ 9\times 9 = 243$$

In [24]:
def fitness(grid):
    score = 0

    for i in range(9):
        score += len(set(grid[i]))  # row uniqueness
        score += len(set(grid[:, i]))  # column uniqueness
    
    # subgrid uniqueness
    for row in range(0, 9, 3):
        for col in range(0, 9, 3):

            subgrid = grid[row:row+3, col:col+3].flatten()
            score += len(set(subgrid))
            
    return score

In [25]:
def initialize_population(puzzle, population_size):
    population = []
    for _ in range(population_size):
        grid = puzzle.copy()
        for i in range(9):
            missing_numbers = list(set(range(1, 10)) - set(grid[i]))
            np.random.shuffle(missing_numbers)
            for j in range(9):
                if grid[i][j] == 0:
                    grid[i][j] = missing_numbers.pop()
        population.append(grid)
    return population

I will start with the most common selection method, which is _tournament selection_. The method works as follows: randomly select a small group of individuals (_tournament size_) and then choose the fittest individual from that group to become a parent for the next generation. This process is repeated multiple times to select a sufficient number of parents for the next generation.

In [26]:
def tournament_selection(population, fitnesses, tournament_size):
    selected = random.sample(range(len(population)), tournament_size)
    best = max(selected, key=lambda idx: fitnesses[idx])
    return population[best]

In [27]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, 8)
    child = parent1.copy()
    child[crossover_point] = parent2[crossover_point].copy()
    return child

def mutate(grid, puzzle, mutation_rate):
    if random.random() < mutation_rate:
        row = random.randint(0, 8)
        col1, col2 = random.sample(range(9), 2)

        while puzzle[row][col1] != 0 or puzzle[row][col2] != 0:
            row = random.randint(0, 8)
            col1, col2 = random.sample(range(9), 2)
        grid[row][col1], grid[row][col2] = grid[row][col2], grid[row][col1]
    return grid

In [28]:
def genetic_algorithm(puzzle, population_size, mutation_rate, tournament_size, generations):
    population = initialize_population(puzzle, population_size)
    best_solution = None
    best_fitness = 0
    
    for generation in range(generations):
        fitnesses = [fitness(grid) for grid in population]
        current_best_fitness = max(fitnesses)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_index = fitnesses.index(current_best_fitness)  
            best_solution = population[best_index]  

        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitnesses, tournament_size)
            parent2 = tournament_selection(population, fitnesses, tournament_size)
            child = crossover(parent1, parent2)
            child = mutate(child, puzzle, mutation_rate)
            new_population.append(child)
        
        population = new_population  

        print(f"Generation {generation} - Best Fitness: {best_fitness}")

        if best_fitness ==  243:
            break

    return best_solution

Consider a 9x9 puzzle defined as follows:

In [29]:
puzzle1 = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

print(f'The fitness of the puzzle is {fitness(puzzle1)}.')

The fitness of the puzzle is 117.


Now let's try to run genetic algorithm with the same parameters as before to see how it performs. 

In [30]:
solution = genetic_algorithm(puzzle1, 100, 0.05, 5, 100)
print(f'The fitness of the solution found is {fitness(solution)}')

Generation 0 - Best Fitness: 207
Generation 1 - Best Fitness: 209
Generation 2 - Best Fitness: 211
Generation 3 - Best Fitness: 212
Generation 4 - Best Fitness: 215
Generation 5 - Best Fitness: 216
Generation 6 - Best Fitness: 217
Generation 7 - Best Fitness: 217
Generation 8 - Best Fitness: 218
Generation 9 - Best Fitness: 218
Generation 10 - Best Fitness: 218
Generation 11 - Best Fitness: 218
Generation 12 - Best Fitness: 218
Generation 13 - Best Fitness: 219
Generation 14 - Best Fitness: 219
Generation 15 - Best Fitness: 219
Generation 16 - Best Fitness: 219
Generation 17 - Best Fitness: 219
Generation 18 - Best Fitness: 219
Generation 19 - Best Fitness: 219
Generation 20 - Best Fitness: 219
Generation 21 - Best Fitness: 219
Generation 22 - Best Fitness: 219
Generation 23 - Best Fitness: 219
Generation 24 - Best Fitness: 220
Generation 25 - Best Fitness: 220
Generation 26 - Best Fitness: 220
Generation 27 - Best Fitness: 220
Generation 28 - Best Fitness: 224
Generation 29 - Best Fit

Although the algorithm increases the fitness of the solution, the provided result is far from the correct solution (with fitness 243), thus the performance for 9x9 grids is not satisfactory. Let's first try to increase the number of generations and see if that improves our results.

In [31]:
solution = genetic_algorithm(puzzle1, 100, 0.05, 5, 500)
print(f'The fitness of the solution found is {fitness(solution)}')

Generation 0 - Best Fitness: 211
Generation 1 - Best Fitness: 214
Generation 2 - Best Fitness: 214
Generation 3 - Best Fitness: 214
Generation 4 - Best Fitness: 217
Generation 5 - Best Fitness: 217
Generation 6 - Best Fitness: 217
Generation 7 - Best Fitness: 218
Generation 8 - Best Fitness: 219
Generation 9 - Best Fitness: 219
Generation 10 - Best Fitness: 220
Generation 11 - Best Fitness: 220
Generation 12 - Best Fitness: 221
Generation 13 - Best Fitness: 221
Generation 14 - Best Fitness: 221
Generation 15 - Best Fitness: 221
Generation 16 - Best Fitness: 221
Generation 17 - Best Fitness: 221
Generation 18 - Best Fitness: 221
Generation 19 - Best Fitness: 222
Generation 20 - Best Fitness: 222
Generation 21 - Best Fitness: 222
Generation 22 - Best Fitness: 224
Generation 23 - Best Fitness: 224
Generation 24 - Best Fitness: 226
Generation 25 - Best Fitness: 226
Generation 26 - Best Fitness: 226
Generation 27 - Best Fitness: 227
Generation 28 - Best Fitness: 227
Generation 29 - Best Fit

The result is slightly better, but we are still far from a solution. To improve our performance, an idea could be to do __hyperparameter tuning__, i.e. defining a set of possible parameters and then understanding under which combination the algorithm performs the best.

### __Hyperparameter Tuning__

In [32]:
def fine_tune_parameters(puzzle, param_grid):
    best_params = None
    best_fitness = 0
    best_solution = None
    
    for population_size in param_grid['population_size']:
        for mutation_rate in param_grid['mutation_rate']:
            for tournament_size in param_grid['tournament_size']:
                for generations in param_grid['generations']:
                    print(f"Testing combination: POPULATION_SIZE={population_size}, MUTATION_RATE={mutation_rate}, TOURNAMENT_SIZE={tournament_size}, GENERATIONS={generations}")
                    solution = genetic_algorithm(puzzle, population_size, mutation_rate, tournament_size, generations)
                    fitness_score = fitness(solution)
                    print(f"Fitness Score: {fitness_score}")
                    
                    if fitness_score > best_fitness:
                        best_fitness = fitness_score
                        best_params = (population_size, mutation_rate, tournament_size, generations)
                        best_solution = solution

    return best_params, best_solution

In [33]:
param_grid = {
    'population_size': [50, 100, 200],
    'mutation_rate': [0.001, 0.01, 0.05, 0.1],
    'tournament_size': [2, 5, 10, 20],
    'generations': [100, 200, 500],
}

best_params, best_solution = fine_tune_parameters(puzzle1, param_grid)
print(f"Best Parameters: {best_params}")
print(f"The best solution is {best_solution} with fitness {fitness(best_solution)}")


Testing combination: POPULATION_SIZE=50, MUTATION_RATE=0.001, TOURNAMENT_SIZE=2, GENERATIONS=100
Generation 0 - Best Fitness: 207
Generation 1 - Best Fitness: 207
Generation 2 - Best Fitness: 211
Generation 3 - Best Fitness: 212
Generation 4 - Best Fitness: 212
Generation 5 - Best Fitness: 213
Generation 6 - Best Fitness: 216
Generation 7 - Best Fitness: 216
Generation 8 - Best Fitness: 216
Generation 9 - Best Fitness: 216
Generation 10 - Best Fitness: 216
Generation 11 - Best Fitness: 216
Generation 12 - Best Fitness: 216
Generation 13 - Best Fitness: 217
Generation 14 - Best Fitness: 217
Generation 15 - Best Fitness: 217
Generation 16 - Best Fitness: 217
Generation 17 - Best Fitness: 217
Generation 18 - Best Fitness: 217
Generation 19 - Best Fitness: 217
Generation 20 - Best Fitness: 217
Generation 21 - Best Fitness: 217
Generation 22 - Best Fitness: 217
Generation 23 - Best Fitness: 217
Generation 24 - Best Fitness: 217
Generation 25 - Best Fitness: 217
Generation 26 - Best Fitness:

The best parameters found are (100, 0.1, 20, 200), yielding higher fitness than before. However, the result is still not a perfect solution.

### __Different Selection Methods__

In [34]:
def rank_selection(population, fitnesses):
    sorted_population = [x for _, x in sorted(zip(fitnesses, population), key=lambda pair: pair[0])]
    n = len(population)
    rank_weights = [i for i in range(1, n+1)]
    total = sum(rank_weights)
    pick = random.uniform(0, total)
    current = 0
    for rank, individual in zip(rank_weights, sorted_population):
        current += rank
        if current > pick:
            return individual

In [35]:
def genetic_algorithm(puzzle, population_size, mutation_rate, generations, selection_method, tournament_size=None):
    population = initialize_population(puzzle, population_size)
    best_solution = None
    best_fitness = 0
    
    for generation in range(generations):
        fitnesses = [fitness(grid) for grid in population]
        current_best_fitness = max(fitnesses)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_index = fitnesses.index(current_best_fitness)  
            best_solution = population[best_index]  

        new_population = []
        for _ in range(population_size):
            if selection_method == "tournament":
                parent1 = tournament_selection(population, fitnesses, tournament_size)
                parent2 = tournament_selection(population, fitnesses, tournament_size)

            if selection_method == "rank":
                parent1 = rank_selection(population, fitnesses)
                parent2 = rank_selection(population, fitnesses)


            child = crossover(parent1, parent2)
            child = mutate(child, puzzle, mutation_rate)
            new_population.append(child)
        
        population = new_population  

        print(f"Generation {generation} - Best Fitness: {best_fitness}")

        if best_fitness ==  243:
            break

    return best_solution

In [36]:
solution_rank = genetic_algorithm(puzzle1, 100, 0.1, 200, "rank") 
solution_tour = genetic_algorithm(puzzle1, 100, 0.1, 200, "tournament", 20)

print(f'The fitness of the solution found by tournament selection is {fitness(solution_tour)}')
print(f'The fitness of the solution found by rank selection is {fitness(solution_rank)}')


Generation 0 - Best Fitness: 206
Generation 1 - Best Fitness: 207
Generation 2 - Best Fitness: 210
Generation 3 - Best Fitness: 211
Generation 4 - Best Fitness: 211
Generation 5 - Best Fitness: 212
Generation 6 - Best Fitness: 215
Generation 7 - Best Fitness: 215
Generation 8 - Best Fitness: 216
Generation 9 - Best Fitness: 220
Generation 10 - Best Fitness: 220
Generation 11 - Best Fitness: 220
Generation 12 - Best Fitness: 220
Generation 13 - Best Fitness: 220
Generation 14 - Best Fitness: 220
Generation 15 - Best Fitness: 221
Generation 16 - Best Fitness: 221
Generation 17 - Best Fitness: 222
Generation 18 - Best Fitness: 222
Generation 19 - Best Fitness: 222
Generation 20 - Best Fitness: 222
Generation 21 - Best Fitness: 222
Generation 22 - Best Fitness: 222
Generation 23 - Best Fitness: 223
Generation 24 - Best Fitness: 223
Generation 25 - Best Fitness: 223
Generation 26 - Best Fitness: 223
Generation 27 - Best Fitness: 223
Generation 28 - Best Fitness: 223
Generation 29 - Best Fit

Considering a different type of selection did not improve our results. However, for the sake of completeness, we now extend the code to consider also other types of selection, namely roulette wheel selection and stochastic universal sampling.

In [37]:
def roulette_wheel_selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual, fitness in zip(population, fitnesses):
        current += fitness
        if current > pick:
            return individual
        
def stochastic_universal_sampling(population, fitnesses):
    total_fitness = sum(fitnesses)
    distance = total_fitness / len(population)
    start = random.uniform(0, distance)
    points = [start + i * distance for i in range(len(population))]
    
    chosen = []
    for p in points:
        current = 0
        for individual, fitness in zip(population, fitnesses):
            current += fitness
            if current > p:
                chosen.append(individual)
                break
    return random.choice(chosen)

In [38]:
def genetic_algorithm(puzzle, population_size, mutation_rate, generations, selection_method, tournament_size=None):
    population = initialize_population(puzzle, population_size)
    best_solution = None
    best_fitness = 0
    
    for generation in range(generations):
        fitnesses = [fitness(grid) for grid in population]
        current_best_fitness = max(fitnesses)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_index = fitnesses.index(current_best_fitness)  
            best_solution = population[best_index]  

        new_population = []
        for _ in range(population_size):
            if selection_method == "tournament":
                parent1 = tournament_selection(population, fitnesses, tournament_size)
                parent2 = tournament_selection(population, fitnesses, tournament_size)

            if selection_method == "rank":
                parent1 = rank_selection(population, fitnesses)
                parent2 = rank_selection(population, fitnesses)

            if selection_method == "roulette":
                parent1 = roulette_wheel_selection(population, fitnesses)
                parent2 = roulette_wheel_selection(population, fitnesses)

            if selection_method == "sus":
                parent1 = stochastic_universal_sampling(population, fitnesses)
                parent2 = stochastic_universal_sampling(population, fitnesses)
                
            child = crossover(parent1, parent2)
            child = mutate(child, puzzle, mutation_rate)
            new_population.append(child)
        
        population = new_population  

        print(f"Generation {generation} - Best Fitness: {best_fitness}")

        if best_fitness ==  243:
            break

    return best_solution

In [39]:
solution_rank = genetic_algorithm(puzzle1, 100, 0.1, 200, "rank")
solution_tour = genetic_algorithm(puzzle1, 100, 0.1, 200, "tournament", 20)
solution_roulette = genetic_algorithm(puzzle1, 100, 0.1, 200, "roulette")
solution_sus = genetic_algorithm(puzzle1, 100, 0.1, 200, "sus") # lower number of generations here because the algorithm is significantly slower

print(f'The fitness of the solution found by tournament selection is {fitness(solution_tour)}')
print(f'The fitness of the solution found by rank selection is {fitness(solution_rank)}')
print(f'The fitness of the solution found by roulette wheel selection is {fitness(solution_roulette)}')
print(f'The fitness of the solution found by stochastic universal sampling is {fitness(solution_sus)}')

Generation 0 - Best Fitness: 209
Generation 1 - Best Fitness: 209
Generation 2 - Best Fitness: 209
Generation 3 - Best Fitness: 210
Generation 4 - Best Fitness: 211
Generation 5 - Best Fitness: 214
Generation 6 - Best Fitness: 214
Generation 7 - Best Fitness: 214
Generation 8 - Best Fitness: 214
Generation 9 - Best Fitness: 214
Generation 10 - Best Fitness: 215
Generation 11 - Best Fitness: 215
Generation 12 - Best Fitness: 216
Generation 13 - Best Fitness: 216
Generation 14 - Best Fitness: 217
Generation 15 - Best Fitness: 217
Generation 16 - Best Fitness: 217
Generation 17 - Best Fitness: 217
Generation 18 - Best Fitness: 217
Generation 19 - Best Fitness: 217
Generation 20 - Best Fitness: 217
Generation 21 - Best Fitness: 217
Generation 22 - Best Fitness: 218
Generation 23 - Best Fitness: 218
Generation 24 - Best Fitness: 218
Generation 25 - Best Fitness: 219
Generation 26 - Best Fitness: 219
Generation 27 - Best Fitness: 219
Generation 28 - Best Fitness: 219
Generation 29 - Best Fit

To possibly improve the analysis one could, of course, include the different options of selection methods in the hyperparameter tuning section. Now we try the same method we used above on different sudoku grids, to see if maybe the issue was with our predefined puzzle. However, since stochastic universal sampling is slow and gives the worst result, we will not consider it anymore.

### __Extension to other puzzles__

In [63]:
puzzle2 = np.array([
    [8, 0, 6, 0, 0, 0, 1, 0, 7],
    [0, 0, 0, 6, 0, 2, 0, 0, 0],
    [0, 5, 3, 0, 0, 4, 8, 0, 6],
    [7, 0, 4, 8, 0, 0, 6, 3, 0],
    [0, 0, 0, 0, 0, 0, 0, 9, 0],
    [1, 0, 0, 5, 0, 0, 4, 0, 0],
    [0, 0, 1, 2, 0, 0, 7, 0, 9],
    [2, 0, 0, 0, 9, 6, 0, 0, 0],
    [0, 7, 0, 0, 1, 0, 0, 8, 0]
])

puzzle3 = np.array([
    [0, 6, 0, 2, 0, 0, 0, 7, 1],
    [4, 0, 5, 0, 0, 0, 0, 0, 2],
    [3, 0, 0, 0, 8, 0, 6, 9, 0],
    [2, 0, 0, 9, 0, 8, 7, 0, 0],
    [0, 9, 3, 0, 0, 0, 8, 0, 0],
    [0, 0, 6, 0, 0, 1, 0, 0, 9],
    [0, 8, 7, 0, 3, 0, 0, 0, 6],
    [6, 0, 0, 0, 0, 0, 5, 0, 7],
    [0, 0, 0, 0, 0, 9, 0, 2, 0]
])

puzzle4 = np.array([
    [0, 2, 3, 4, 5, 6, 7, 8, 0],
    [4, 0, 6, 7, 0, 9, 1, 2, 3],
    [7, 8, 9, 1, 2, 3, 4, 5, 6],
    [2, 0, 4, 3, 6, 5, 8, 9, 7],
    [3, 6, 5, 0, 9, 7, 2, 1, 4],
    [8, 9, 7, 2, 0, 4, 3, 0, 5],
    [5, 3, 1, 6, 4, 2, 9, 0, 8],
    [6, 4, 2, 9, 7, 0, 5, 3, 1],
    [0, 7, 8, 5, 3, 1, 6, 4, 2]
])

puzzle5 = np.array([
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 7, 0, 0, 0, 6, 0],
    [4, 0, 0, 0, 8, 0, 3, 0, 0],
    [7, 0, 0, 0, 2, 6, 0, 0, 0],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
])


In [64]:
sudoku_dict = {'sudoku 2': puzzle2, 'sudoku 3': puzzle3, 'sudoku 4': puzzle4, 'sudoku 5': puzzle5}
for i, puzzle in sudoku_dict.items():
    print(f'{i} has fitness {fitness(puzzle)}')

sudoku 2 has fitness 117
sudoku 3 has fitness 117
sudoku 4 has fitness 234
sudoku 5 has fitness 114


We see that sudoku 4 is clearly closer to being solved than the other sudokus.

In [43]:
fitnesses_dict = {}

for i, puzzle in sudoku_dict.items():
    solution_rank = genetic_algorithm(puzzle, 100, 0.1, 200, "rank")
    solution_tour = genetic_algorithm(puzzle, 100, 0.1, 200, "tournament", 20)
    solution_roulette = genetic_algorithm(puzzle, 100, 0.1, 200, "roulette")

    fitnesses = [fitness(solution_rank), fitness(solution_tour), fitness(solution_roulette)]
    fitnesses_dict[i] = fitnesses

for i in sudoku_dict.keys():
    print(f'Solutions of {i} have fitnesses: {fitnesses_dict[i]}')


Generation 0 - Best Fitness: 203
Generation 1 - Best Fitness: 206
Generation 2 - Best Fitness: 210
Generation 3 - Best Fitness: 210
Generation 4 - Best Fitness: 211
Generation 5 - Best Fitness: 212
Generation 6 - Best Fitness: 215
Generation 7 - Best Fitness: 215
Generation 8 - Best Fitness: 215
Generation 9 - Best Fitness: 215
Generation 10 - Best Fitness: 215
Generation 11 - Best Fitness: 215
Generation 12 - Best Fitness: 215
Generation 13 - Best Fitness: 215
Generation 14 - Best Fitness: 216
Generation 15 - Best Fitness: 216
Generation 16 - Best Fitness: 218
Generation 17 - Best Fitness: 218
Generation 18 - Best Fitness: 219
Generation 19 - Best Fitness: 219
Generation 20 - Best Fitness: 220
Generation 21 - Best Fitness: 220
Generation 22 - Best Fitness: 221
Generation 23 - Best Fitness: 221
Generation 24 - Best Fitness: 221
Generation 25 - Best Fitness: 221
Generation 26 - Best Fitness: 223
Generation 27 - Best Fitness: 223
Generation 28 - Best Fitness: 223
Generation 29 - Best Fit

We happily see that the algorithm has solved the easy sudoku. On the other hand, the closest it got to a solution for the other three puzzles was with tournament selection for puzzle2. Let's see if we can remove some cells from puzzle4 and still keep it solvable.

In [65]:
for i in range(1, puzzle4.shape[0]):
    nonzero_indices = np.nonzero(puzzle4[i])[0]

    random_index = np.random.choice(nonzero_indices)

    modified_puzzle = puzzle4
    modified_puzzle[i, random_index] = 0

print(modified_puzzle)
fitness(modified_puzzle)


[[0 2 3 4 5 6 7 8 0]
 [0 0 6 7 0 9 1 2 3]
 [7 8 9 1 2 0 4 5 6]
 [2 0 4 0 6 5 8 9 7]
 [3 6 0 0 9 7 2 1 4]
 [8 9 7 2 0 4 0 0 5]
 [5 3 0 6 4 2 9 0 8]
 [6 4 2 9 0 0 5 3 1]
 [0 0 8 5 3 1 6 4 2]]


213

In [66]:
solution = genetic_algorithm(modified_puzzle, 100, 0.1, 200, "tournament", 20)
fitness(solution)

Generation 0 - Best Fitness: 239
Generation 1 - Best Fitness: 241
Generation 2 - Best Fitness: 243


243

In [67]:
for i in range(1, puzzle4.shape[0]):
    nonzero_indices = np.nonzero(puzzle4[i])[0]

    random_index = np.random.choice(nonzero_indices)

    modified_puzzle[i, random_index] = 0

print(modified_puzzle)
fitness(modified_puzzle)


[[0 2 3 4 5 6 7 8 0]
 [0 0 6 7 0 9 0 2 3]
 [7 8 9 1 2 0 4 5 0]
 [2 0 4 0 6 5 0 9 7]
 [3 0 0 0 9 7 2 1 4]
 [8 9 7 2 0 4 0 0 0]
 [5 0 0 6 4 2 9 0 8]
 [6 4 2 9 0 0 0 3 1]
 [0 0 8 5 3 1 6 4 0]]


189

In [68]:
solution = genetic_algorithm(modified_puzzle, 100, 0.1, 200, "tournament", 20)
fitness(solution)

Generation 0 - Best Fitness: 226
Generation 1 - Best Fitness: 231
Generation 2 - Best Fitness: 235
Generation 3 - Best Fitness: 238
Generation 4 - Best Fitness: 241
Generation 5 - Best Fitness: 241
Generation 6 - Best Fitness: 241
Generation 7 - Best Fitness: 243


243

Again, it finds the solution quickly enough.

In [69]:
for i in range(1, puzzle4.shape[0]):
    nonzero_indices = np.nonzero(puzzle4[i])[0]

    random_index = np.random.choice(nonzero_indices)

    modified_puzzle[i, random_index] = 0

print(modified_puzzle)
fitness(modified_puzzle)

[[0 2 3 4 5 6 7 8 0]
 [0 0 0 7 0 9 0 2 3]
 [7 0 9 1 2 0 4 5 0]
 [2 0 4 0 6 5 0 9 0]
 [3 0 0 0 9 0 2 1 4]
 [8 9 7 2 0 0 0 0 0]
 [0 0 0 6 4 2 9 0 8]
 [6 4 2 0 0 0 0 3 1]
 [0 0 8 5 3 0 6 4 0]]


165

In [70]:
solution = genetic_algorithm(modified_puzzle, 100, 0.1, 200, "tournament", 20)
fitness(solution)

Generation 0 - Best Fitness: 220
Generation 1 - Best Fitness: 223
Generation 2 - Best Fitness: 224
Generation 3 - Best Fitness: 227
Generation 4 - Best Fitness: 228
Generation 5 - Best Fitness: 229
Generation 6 - Best Fitness: 230
Generation 7 - Best Fitness: 231
Generation 8 - Best Fitness: 231
Generation 9 - Best Fitness: 231
Generation 10 - Best Fitness: 233
Generation 11 - Best Fitness: 234
Generation 12 - Best Fitness: 234
Generation 13 - Best Fitness: 234
Generation 14 - Best Fitness: 235
Generation 15 - Best Fitness: 235
Generation 16 - Best Fitness: 236
Generation 17 - Best Fitness: 236
Generation 18 - Best Fitness: 237
Generation 19 - Best Fitness: 237
Generation 20 - Best Fitness: 237
Generation 21 - Best Fitness: 241
Generation 22 - Best Fitness: 241
Generation 23 - Best Fitness: 241
Generation 24 - Best Fitness: 241
Generation 25 - Best Fitness: 241
Generation 26 - Best Fitness: 241
Generation 27 - Best Fitness: 241
Generation 28 - Best Fitness: 241
Generation 29 - Best Fit

243

In [71]:
for i in range(1, puzzle4.shape[0]):
    nonzero_indices = np.nonzero(puzzle4[i])[0]

    random_index = np.random.choice(nonzero_indices)

    modified_puzzle[i, random_index] = 0

print(modified_puzzle)
fitness(modified_puzzle)

[[0 2 3 4 5 6 7 8 0]
 [0 0 0 7 0 9 0 0 3]
 [7 0 9 1 2 0 4 0 0]
 [2 0 4 0 6 5 0 0 0]
 [0 0 0 0 9 0 2 1 4]
 [8 9 0 2 0 0 0 0 0]
 [0 0 0 6 4 2 0 0 8]
 [6 0 2 0 0 0 0 3 1]
 [0 0 8 0 3 0 6 4 0]]


141

In [72]:
solution = genetic_algorithm(modified_puzzle, 100, 0.1, 200, "tournament", 20)
fitness(solution)

Generation 0 - Best Fitness: 213
Generation 1 - Best Fitness: 217
Generation 2 - Best Fitness: 221
Generation 3 - Best Fitness: 224
Generation 4 - Best Fitness: 225
Generation 5 - Best Fitness: 225
Generation 6 - Best Fitness: 226
Generation 7 - Best Fitness: 226
Generation 8 - Best Fitness: 227
Generation 9 - Best Fitness: 227
Generation 10 - Best Fitness: 227
Generation 11 - Best Fitness: 227
Generation 12 - Best Fitness: 229
Generation 13 - Best Fitness: 231
Generation 14 - Best Fitness: 231
Generation 15 - Best Fitness: 233
Generation 16 - Best Fitness: 233
Generation 17 - Best Fitness: 233
Generation 18 - Best Fitness: 233
Generation 19 - Best Fitness: 233
Generation 20 - Best Fitness: 233
Generation 21 - Best Fitness: 233
Generation 22 - Best Fitness: 234
Generation 23 - Best Fitness: 234
Generation 24 - Best Fitness: 236
Generation 25 - Best Fitness: 236
Generation 26 - Best Fitness: 236
Generation 27 - Best Fitness: 236
Generation 28 - Best Fitness: 236
Generation 29 - Best Fit

243

In [73]:
for i in range(1, puzzle4.shape[0]):
    nonzero_indices = np.nonzero(puzzle4[i])[0]

    random_index = np.random.choice(nonzero_indices)

    modified_puzzle[i, random_index] = 0

print(modified_puzzle)
fitness(modified_puzzle)

[[0 2 3 4 5 6 7 8 0]
 [0 0 0 7 0 0 0 0 3]
 [7 0 0 1 2 0 4 0 0]
 [2 0 4 0 0 5 0 0 0]
 [0 0 0 0 9 0 0 1 4]
 [8 0 0 2 0 0 0 0 0]
 [0 0 0 6 4 2 0 0 0]
 [6 0 2 0 0 0 0 0 1]
 [0 0 8 0 3 0 6 0 0]]


117

In [74]:
solution = genetic_algorithm(modified_puzzle, 100, 0.1, 200, "tournament", 20)
fitness(solution)

Generation 0 - Best Fitness: 209
Generation 1 - Best Fitness: 212
Generation 2 - Best Fitness: 215
Generation 3 - Best Fitness: 218
Generation 4 - Best Fitness: 219
Generation 5 - Best Fitness: 220
Generation 6 - Best Fitness: 220
Generation 7 - Best Fitness: 221
Generation 8 - Best Fitness: 223
Generation 9 - Best Fitness: 224
Generation 10 - Best Fitness: 227
Generation 11 - Best Fitness: 231
Generation 12 - Best Fitness: 231
Generation 13 - Best Fitness: 231
Generation 14 - Best Fitness: 232
Generation 15 - Best Fitness: 232
Generation 16 - Best Fitness: 232
Generation 17 - Best Fitness: 232
Generation 18 - Best Fitness: 232
Generation 19 - Best Fitness: 232
Generation 20 - Best Fitness: 232
Generation 21 - Best Fitness: 232
Generation 22 - Best Fitness: 232
Generation 23 - Best Fitness: 232
Generation 24 - Best Fitness: 235
Generation 25 - Best Fitness: 235
Generation 26 - Best Fitness: 235
Generation 27 - Best Fitness: 236
Generation 28 - Best Fitness: 236
Generation 29 - Best Fit

239

Now, the solver starts struggling. However, we can still conclude that, for a grid which is easy enough (i.e. close enough to being solved), the sudoku solver works.

## __Conclusions__
As mentioned in the introduction, genetic algorithms are not the best approach to solve sudokus, as it has been broadly explored in literature. As a matter of fact, the goal of this project was not to build an efficient solver, rather to try and understand what improvements could be made to get closer to finding a solution and to explore different parameters and methods to see how they influence the final outcome.<br>

In the end, with this brief analysis, we have been able to solve easy enough puzzles (i.e. either 4x4 and 6x6 grids or 9x9 grids close enough to the solution), we have seen that typically tournament selection works better than the other approaches and that for harder puzzles genetic algorithms tend to get stuck at some suboptimal value. In principle, one could think that the algorithm gets stuck at a local minimum and can't get out of it. However, even employing extremely high mutation rates, which should help the algorithm explore the solution space as much as possible, the result's don't change. Thus, this might be a problem caused by the intrinsic structure of the puzzles, or by how we defined the framework. <br>

Despite having obtained some successful results, further improvements could be done on this analysis. For example, one could add adaptive mutation rate or change crossover rate. In particular, we could also try different types of crossover or different types of mutations. Finally, different representation of sudoku grids could be employed to see if that affects the performance as well.